In [1]:
!python3 -m pip install xmltodict numpy pandas matplotlib mne scikit-learn scipy joblib autoreject tqdm PyWavelets spectrum xgboost seaborn mock plotly

     |████████████████████████████████| 6.4MB 5.7MB/s 
     |████████████████████████████████| 235kB 55.4MB/s 
  Created wheel for spectrum: filename=spectrum-0.7.6-cp36-cp36m-linux_x86_64.whl size=234245 sha256=ca263f0f30a3015aea05c0f864aa0ad756fe410194debf0970dbaf74e7529ea9
  Stored in directory: /root/.cache/pip/wheels/7b/a1/1f/16e3bd0418dc16201a4f2e696ab00de3e3c95549cba7df5d13
Successfully built spectrum


In [0]:
#%% #* Import statements
import os
import numpy as np
import pandas as pd
import scipy.stats as sp
import matplotlib.pyplot as plt
import mne
from tqdm import tqdm as tqdm
from spectrum import arburg
from autoreject import AutoReject
import mock
import sklearn.preprocessing as skpr

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
%matplotlib inline

In [0]:
DATA_DIR = '/content/drive/My Drive/mahnob/Sessions'

def getDataFiles(rootFolder):
  sessionFolders, sessions = list(sorted(os.listdir(rootFolder))), []
  for sessionFolder in sessionFolders:
    _path = os.path.join(DATA_DIR, sessionFolder)
    if os.path.isdir(_path):
      node = {'folder': sessionFolder}
      for subfile in os.listdir(_path):
        if subfile.endswith('.bdf'):
          node['bdf'] = subfile
        elif subfile.endswith('.xml'):
          node['xml'] = subfile
      if 'bdf' in node and 'xml' in node and 'S_Trial' in node['bdf']:
        sessions.append(node)
  return sessions

In [0]:
def concat_features(save_as):
  inp = pd.DataFrame()
  sessions = getDataFiles(DATA_DIR)
  for session in tqdm(sessions):
    each = pd.read_pickle(
        os.path.join(DATA_DIR, session['folder'], 'features.pkl'))
    inp = pd.concat([inp, each])
  inp.to_pickle(os.path.join(DATA_DIR, save_as))

In [0]:
## Run this for concating the features from each Session
# concat_features('data_935.pkl')

In [0]:
inp = pd.DataFrame(pd.read_pickle(os.path.join(DATA_DIR, 'data_935_noBase.pkl'))).dropna()

In [0]:
output_labels = ['valence', 'arousal', 'control', 'prediction', 'emotion']
emo_dims = output_labels[:-1]

inp_emotion = output_labels[-1]
emo_dims_sub_id = emo_dims + ['subjectid']
inp_emo_dims = inp[emo_dims_sub_id]
inp_emotion = inp['emotion']
inp = inp.drop(['stim_video', 'emotion'] + emo_dims_sub_id, axis=1)

# inp = (inp - inp.mean()) / inp.std()

# inp_emo_dims = inp_emo_dims.groupby('subjectid').transform(lambda x: (x - x.mean()) / x.std())

inp_emo_dims = inp_emo_dims.drop('subjectid', axis=1)

inp[emo_dims] = inp_emo_dims
inp['emotion'] = inp_emotion

In [17]:
inp

,ch0_coeff_var,ch0_kurtosis,ch0_skew,ch0_d1_mean,ch0_d1_max,ch0_d2_mean,ch0_d2_max,ch0_ar1,ch0_ar2,ch0_ar3,ch0_hjworth_activity,ch0_hjworth_mobility,ch0_hjworth_complexity,ch0_PMax1,ch0_PMax2,ch0_PMax3,ch0_PMax4,ch0_PRatio1,ch0_PRatio2,ch0_PRatio3,ch0_PRatio4,ch0_cA_mean,ch0_cA_std,ch0_cA_energy,ch0_cA_entropy,ch0_cD_mean,ch0_cD_std,ch0_cD_energy,ch0_cD_entropy,ch1_coeff_var,ch1_kurtosis,ch1_skew,ch1_d1_mean,ch1_d1_max,ch1_d2_mean,ch1_d2_max,ch1_ar1,ch1_ar2,ch1_ar3,ch1_hjworth_activity,...,ch30_cA_energy,ch30_cA_entropy,ch30_cD_mean,ch30_cD_std,ch30_cD_energy,ch30_cD_entropy,ch31_coeff_var,ch31_kurtosis,ch31_skew,ch31_d1_mean,ch31_d1_max,ch31_d2_mean,ch31_d2_max,ch31_ar1,ch31_ar2,ch31_ar3,ch31_hjworth_activity,ch31_hjworth_mobility,ch31_hjworth_complexity,ch31_PMax1,ch31_PMax2,ch31_PMax3,ch31_PMax4,ch31_PRatio1,ch31_PRatio2,ch31_PRatio3,ch31_PRatio4,ch31_cA_mean,ch31_cA_std,ch31_cA_energy,ch31_cA_entropy,ch31_cD_mean,ch31_cD_std,ch31_cD_energy,ch31_cD_entropy,valence,arousal,control,prediction,emotion
0,-0.001913,15.005469,3.476950,-0.011978,14.562473,-0.005426,14.937472,-1.779246,1.121202,-0.341956,599.829847,0.118040,7.759557,56.013802,4.604899,1.921906,0.698087,12.163958,29.144920,6.596458,31.540927,-18103.822061,34.528159,1.265113e+11,2.480605e+12,-0.010085,1.298883,651.259105,917.112162,-0.001767,5.531191,1.785631,0.008801,9.406233,-0.004243,10.093731,-1.668597,1.110092,-0.441495,190.687747,...,1.870033e+10,3.309209e+11,-0.005350,1.440423,800.890755,1034.567670,-0.000610,0.303626,-0.255701,-0.003830,6.531238,-0.004161,7.093737,-1.550065,1.147388,-0.597322,60.101021,0.341077,3.358366,18.726083,2.229324,0.767103,0.577095,8.399893,24.411443,3.863012,27.317604,-17960.509795,10.830781,1.245159e+11,2.439498e+12,-0.010807,1.524132,896.714887,1268.285437,3,4,1,5,2
1,-0.001343,-1.135068,0.529747,0.064537,7.749986,0.001428,6.218739,-1.582634,1.176391,-0.593757,295.110672,0.127695,8.815461,11.395320,1.679457,1.041164,0.324498,6.785123,10.944792,5.175556,12.557849,-18086.614697,24.304540,1.262707e+11,2.475646e+12,-0.010636,1.298609,650.987985,727.979486,-0.001171,-0.923715,0.051946,-0.011693,6.437488,0.000367,6.312488,-1.600242,1.217304,-0.617062,84.237546,...,1.913417e+10,3.390368e+11,0.012835,1.364439,718.677728,821.029500,-0.000494,-0.246971,0.022410,0.020697,6.374988,0.004447,8.624984,-1.557724,1.205694,-0.647970,39.278756,0.411117,2.798031,6.234830,1.627520,1.327490,0.606406,3.830877,4.696704,2.683877,5.922717,-17948.735199,8.760436,1.243527e+11,2.436137e+12,0.004754,1.441516,802.105070,1023.509482,3,4,1,5,2
2,-0.000619,-0.326046,0.078431,0.025098,9.062483,0.006813,12.531227,-1.611264,1.240038,-0.628773,62.314316,0.294762,3.791638,8.899150,2.107174,1.418718,0.341531,4.223262,6.272669,6.169798,7.757936,-18038.565541,11.105047,1.256005e+11,2.461838e+12,-0.016049,1.321130,673.818129,849.430604,-0.000834,-0.319717,0.498362,-0.005786,7.437486,0.005304,8.718734,-1.608750,1.271197,-0.662447,42.767924,...,1.902923e+10,3.370725e+11,0.002406,1.561548,941.237057,1331.858667,-0.001130,-0.927706,0.239413,-0.050521,6.687488,0.008404,8.843734,-1.579043,1.260455,-0.681412,206.177425,0.187561,6.104228,9.604177,1.249670,1.037440,0.420110,7.685370,9.257571,2.974628,10.462141,-17971.977491,20.300307,1.246751e+11,2.442776e+12,0.005473,1.538929,914.176045,1311.529410,3,4,1,5,2
3,-0.000499,-0.081596,0.236142,-0.003748,9.312483,0.003549,8.687484,-1.550907,1.072857,-0.521950,40.399920,0.336023,3.359292,7.204403,2.137508,1.118008,0.905036,3.370468,6.443961,2.361792,8.355850,-18026.469004,8.901400,1.254321e+11,2.458368e+12,0.000015,1.165802,524.610082,546.059947,-0.000956,-0.325145,0.198168,0.018986,5.281240,0.001836,6.499988,-1.604430,1.169984,-0.565554,56.102424,...,1.910564e+10,3.385025e+11,-0.013561,1.488405,855.195742,1163.588353,-0.000450,-0.243609,-0.055805,-0.030598,6.531238,-0.005181,7.781236,-1.483826,1.126890,-0.643064,32.845070,0.452726,2.621704,3.783938,1.778027,1.032215,0.305153,2.128167,3.665843,5.826680,5.388378,-18012.645851,8.008575,1.25

In [18]:
px.scatter_matrix(inp,
    dimensions=['valence', 'arousal', 'control', 'prediction'], color='emotion')

In [19]:
px.imshow(inp[['valence', 'arousal', 'control', 'prediction']].corr())

In [0]:
output_labels = ['valence', 'arousal', 'control', 'prediction', 'emotion']
emo_dims = output_labels[:-1]
Y = inp[output_labels]
X = inp.drop(columns=output_labels)

In [0]:
X2 = pd.DataFrame()
size_X2 = int(len(X.columns) / 32)
for i in range(size_X2):
  X2['_'.join(X.columns[i].split('_')[1:])] = X[X.columns[i::size_X2]].mean(axis=1)

In [0]:
minmax_dims = ['ar1', 'ar2', 'ar3']
minmax_dims += [f'{c}_{feat}' for c in ['cA', 'cD'] for feat in ['mean', 'std', 'energy', 'entropy']]

In [0]:
for dim in minmax_dims:
  X2[f'{dim}_min'] = X[[f'ch{chan}_{dim}' for chan in range(32)]].min(axis=1)
  X2[f'{dim}_max'] = X[[f'ch{chan}_{dim}' for chan in range(32)]].max(axis=1)

In [24]:
X2

,coeff_var,kurtosis,skew,d1_mean,d1_max,d2_mean,d2_max,ar1,ar2,ar3,hjworth_activity,hjworth_mobility,hjworth_complexity,PMax1,PMax2,PMax3,PMax4,PRatio1,PRatio2,PRatio3,PRatio4,cA_mean,cA_std,cA_energy,cA_entropy,cD_mean,cD_std,cD_energy,cD_entropy,ar1_min,ar1_max,ar2_min,ar2_max,ar3_min,ar3_max,cA_mean_min,cA_mean_max,cA_std_min,cA_std_max,cA_energy_min,cA_energy_max,cA_entropy_min,cA_entropy_max,cD_mean_min,cD_mean_max,cD_std_min,cD_std_max,cD_energy_min,cD_energy_max,cD_entropy_min,cD_entropy_max
0,-0.000817,1.385376,0.153950,-0.004437,8.368149,-0.003784,9.529279,-1.516150,1.037651,-0.521501,217.933527,0.199901,6.520618,19.300715,1.829833,1.083941,0.481631,11.901141,18.603130,3.787577,20.382355,-12821.133495,19.282207,1.130111e+11,2.246619e+12,-0.003142,1.392032,836.242621,1405.269353,-1.779246,-1.190684,0.646605,1.565949,-0.907205,-0.309605,-31793.603170,14739.388232,5.663456,39.774956,2.554023e+09,3.901820e+11,4.011125e+10,8.090051e+12,-0.024549,0.023169,0.648588,2.999578,162.412349,3473.126215,-28.346230,8937.473975
1,-0.000605,-0.343304,0.334797,-0.013156,6.696277,0.001823,8.093735,-1.494602,1.062716,-0.568113,108.724995,0.263973,4.928193,10.452956,1.494933,0.906143,0.420375,8.160810,11.943047,3.719450,13.555694,-12846.660013,13.617285,1.133107e+11,2.252982e+12,0.000936,1.356697,798.689180,1274.774878,-1.757999,-1.172010,0.652676,1.564404,-0.916241,-0.362611,-31869.607386,14702.752900,5.725770,27.284464,2.531537e+09,3.920495e+11,3.973567e+10,8.130643e+12,-0.039294,0.029792,0.595335,3.025144,136.833015,3532.597607,-48.979698,9235.384020
2,-0.000651,-0.494756,0.142554,0.016472,7.824204,0.005648,8.325180,-1.536925,1.137604,-0.600679,135.138115,0.265776,5.155265,9.056223,1.683857,1.152212,0.457497,6.345640,8.262278,4.113523,9.740471,-12841.613508,14.656776,1.132730e+11,2.252110e+12,-0.004128,1.409607,855.834945,1417.995086,-1.738083,-1.281181,0.752656,1.586035,-0.913852,-0.397356,-31822.091004,14705.079831,5.267933,38.978936,2.522294e+09,3.908814e+11,3.958136e+10,8.105252e+12,-0.056521,0.029740,0.647855,3.021783,162.058955,3524.632847,-37.630996,9207.943638
3,-0.000637,-0.409854,-0.250486,-0.002587,7.254869,-0.002355,8.533187,-1.488197,1.053504,-0.565307,128.923273,0.273854,5.178132,7.616664,1.865179,0.920780,0.451719,4.918307,9.941882,4.234905,12.068605,-12826.782277,14.079137,1.130886e+11,2.248334e+12,-0.001913,1.368587,826.526747,1372.799625,-1.732677,-1.253546,0.698738,1.558658,-0.917893,-0.354796,-31838.200919,14708.344069,4.811251,34.108953,2.495521e+09,3.912774e+11,3.913459e+10,8.113860e+12,-0.023708,0.025562,0.642069,3.128015,159.133673,3776.963196,-27.382854,10195.642259
4,-0.000976,-0.469670,-0.026053,-0.042189,7.947251,-0.001305,8.332992,-1.502020,1.062290,-0.560270,189.579325,0.212530,6.833778,25.551277,2.408184,0.904731,0.455597,12.761183,27.083838,6.731572,29.675698,-12848.585745,17.867863,1.134440e+11,2.255876e+12,-0.004857,1.330958,775.461205,1253.169827,-1.709642,-1.215146,0.670793,1.563960,-0.915898,-0.363738,-31911.649445,14718.878426,5.346426,39.169386,2.479611e+09,3.930845e+11,3.886926e+10,8.153145e+12,-0.041841,0.019457,0.634201,3.073831,155.282753,3647.243411,-35.240408,9626.034285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,0.038928,1.096667,0.416314,0.010789,17.051726,0.003682,19.338831,-1.350986,0.870823,-0.518961,356.811154,0.429280,3.785886,43.621591,5.829719,1.706263,1.373666,6.436685,16.443872,3.249864,18.893754,-6285.842161,19.777957,5.007730e+10,9.836932e+11,-0.002033,3.102649,6809.913428,32009.550638,-1.675675,-0.991839,0.377806,1.665522,-0.998196,-0.270871,-30098.990758,16285.552799,1.024750,89.139289,3.863080e+04,3.496967e+11,1.971775e+05,7.212319e+12,-0.079634,0.075379,0.351329,16.628778,47.644739,106735.547296,-62.542062,633075.795339
28,0.190341,1.930401,0.656686,-0.009539,15.018527,-0.000904,19.556604,-1.322248,0.849287,-0

In [0]:
Y2 = pd.DataFrame()
for emo_dim in emo_dims:
  Y2[emo_dim] = Y.apply(lambda row: 'low' if row[emo_dim] <= 4.5 else 'high', axis=1)

In [0]:
def get_model(X, Y):
  model_fi = []
  for emotion in emo_dims:
    print('\n\t+++ Model for {}'.format(emotion))
    y = Y[emotion]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBClassifier()
    model.fit(X_train, y_train)
    predict_train = model.predict(X_train)
    accuracy_train = accuracy_score(y_train, predict_train)
    print('accuracy_score on train dataset : ', accuracy_train)
    predict_test = model.predict(X_test)
    accuracy_test = accuracy_score(y_test, predict_test)
    print('accuracy_score on test dataset : ', accuracy_test)
    model_fi.append(pd.Series(model.feature_importances_, index=X.columns, name=emotion))
  feat_importances = pd.concat(model_fi, axis=1)
  return model, feat_importances

In [0]:
def plot_feat_imp(feat_importances, show_fig=True):
  fig = make_subplots(rows=2, cols=2)
  for i in range(2):
    for j in range(2):
      ser = feat_importances[emo_dims[i * 2 + j]].sort_values(ascending=False).nlargest(10)
      fig.add_trace(go.Bar(
          x=ser.keys(),
          y=ser,
          name=emo_dims[i * 2 + j]
      ), row = i + 1, col = j + 1)
      # fig.update_xaxes(title_text="importance", row=i + 1, col=j + 1)
      # fig.update_yaxes(title_text="feature", row=i + 1, col=j + 1)
  fig.update_layout(title_text="Feature Importance", height=1000)
  if show_fig:
    fig.show()
  else:
    return fig

In [28]:
model, feat_importances = get_model(X2, Y2)


	+++ Model for valence
accuracy_score on train dataset :  0.8055365101131299
accuracy_score on test dataset :  0.7764826876928351

	+++ Model for arousal
accuracy_score on train dataset :  0.8039938292766541
accuracy_score on test dataset :  0.7805965032567707

	+++ Model for control
accuracy_score on train dataset :  0.8225059993143641
accuracy_score on test dataset :  0.792595131984916

	+++ Model for prediction
accuracy_score on train dataset :  0.8129928008227632
accuracy_score on test dataset :  0.8138498457319163


In [29]:
plot_feat_imp(feat_importances)

In [0]:
X2

,coeff_var,kurtosis,skew,d1_mean,d1_max,d2_mean,d2_max,ar1,ar2,ar3,hjworth_activity,hjworth_mobility,hjworth_complexity,PMax1,PMax2,PMax3,PMax4,PRatio1,PRatio2,PRatio3,PRatio4,cA_mean,cA_std,cA_energy,cA_entropy,cD_mean,cD_std,cD_energy,cD_entropy,ar1_min,ar1_max,ar2_min,ar2_max,ar3_min,ar3_max,cA_mean_min,cA_mean_max,cA_std_min,cA_std_max,cA_energy_min,cA_energy_max,cA_entropy_min,cA_entropy_max,cD_mean_min,cD_mean_max,cD_std_min,cD_std_max,cD_energy_min,cD_energy_max,cD_entropy_min,cD_entropy_max
0,-0.148941,1.385376,0.153950,-0.004437,8.368149,-0.003784,9.529279,-1.514540,1.036467,-0.519901,217.933527,0.235994,4.503993,154.405719,10.562634,0.317756,0.173466,28.002255,626.706388,187.000315,680.765700,99.593174,19.282207,7.995540e+06,8.417833e+07,-0.003142,1.392032,836.242621,1405.269353,-1.778835,-1.190270,0.646496,1.565566,-0.906585,-0.309159,-111.164267,302.053939,5.663456,39.774956,28694.196881,3.532456e+07,151630.502936,4.037710e+08,-0.024549,0.023169,0.648588,2.999578,162.412349,3473.126215,-28.346230,8937.473975
1,-1.192374,-0.343304,0.334797,-0.013156,6.696277,0.001823,8.093735,-1.493038,1.061751,-0.566486,108.724995,0.327882,3.094050,83.623646,11.025561,0.285827,0.144937,15.374779,433.370546,201.621695,500.008585,74.066656,13.617285,6.165888e+06,6.350578e+07,0.000936,1.356697,798.689180,1274.774878,-1.757927,-1.171861,0.652634,1.561962,-0.910928,-0.362547,-137.744126,276.828931,5.725770,27.284464,37981.168275,2.970339e+07,200441.934597,3.344365e+08,-0.039294,0.029792,0.595335,3.025144,136.833015,3532.597607,-48.979698,9235.384020
2,0.014314,-0.494756,0.142554,0.016472,7.824204,0.005648,8.325180,-1.535013,1.136431,-0.598854,135.138115,0.315404,3.377651,72.449782,11.728601,0.325193,0.130765,13.645512,288.650186,181.835661,342.746829,79.113161,14.656776,6.001434e+06,6.151886e+07,-0.004128,1.409607,855.834945,1417.995086,-1.738007,-1.281036,0.748732,1.584505,-0.911472,-0.397316,-136.709028,240.174545,5.267933,38.978936,38881.710060,2.233597e+07,200174.552839,2.450747e+08,-0.056521,0.029740,0.647855,3.021783,162.058955,3524.632847,-37.630996,9207.943638
3,0.148039,-0.409854,-0.250486,-0.002587,7.254869,-0.002355,8.533187,-1.487350,1.053011,-0.564446,128.923273,0.340632,3.172374,60.933308,10.944112,0.306309,0.131431,11.241041,260.040052,210.621152,308.428107,93.944392,14.079137,5.939305e+06,6.055543e+07,-0.001913,1.368587,826.526747,1372.799625,-1.732608,-1.253525,0.697976,1.558333,-0.917517,-0.354647,-82.284666,236.298376,4.811251,34.108953,51875.185088,2.167265e+07,278521.325929,2.372422e+08,-0.023708,0.025562,0.642069,3.128015,159.133673,3776.963196,-27.382854,10195.642259
4,0.140779,-0.469670,-0.026053,-0.042189,7.947251,-0.001305,8.332992,-1.501220,1.061810,-0.559516,189.579325,0.268223,4.071339,204.723646,9.722782,0.331290,0.127602,35.912553,640.031108,228.259682,676.665618,72.140924,17.867863,4.242848e+06,4.216789e+07,-0.004857,1.330958,775.461205,1253.169827,-1.709482,-1.215033,0.669656,1.561967,-0.913903,-0.363606,-78.000841,243.762706,5.346426,39.169386,140536.228367,2.300959e+07,892889.710786,2.531496e+08,-0.041841,0.019457,0.634201,3.073831,155.282753,3647.243411,-35.240408,9626.034285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,1.366056,1.096667,0.416314,0.010789,17.051726,0.003682,19.338831,-1.344550,0.868985,-0.512350,356.811154,1.234323,1.288894,348.980970,116.684078,1.720213,0.373109,33.043537,237.529914,1175.566113,359.745926,31.861974,19.777957,1.898644e+06,1.744942e+07,-0.002033,3.102649,6809.913428,32009.550638,-1.674550,-0.948480,0.377742,1.666478,-0.997192,-0.242830,-101.319336,149.053325,1.024750,89.139289,28007.189879,8.636579e+06,141035.395644,8.662275e+07,-0.079634,0.075379,0.351329,16.628778,47.644739,106735.547296,-62.542062,633075.795339
28,0.257526,1.930401,0.656686,-0.009539,15.018527,-0.000904,19.556604,-1.317751,0.848833,-0.5

In [0]:
list(sorted(X2.columns))

['PMax1',
 'PMax2',
 'PMax3',
 'PMax4',
 'PRatio1',
 'PRatio2',
 'PRatio3',
 'PRatio4',
 'ar1',
 'ar1_max',
 'ar1_min',
 'ar2',
 'ar2_max',
 'ar2_min',
 'ar3',
 'ar3_max',
 'ar3_min',
 'cA_energy',
 'cA_energy_max',
 'cA_energy_min',
 'cA_entropy',
 'cA_entropy_max',
 'cA_entropy_min',
 'cA_mean',
 'cA_mean_max',
 'cA_mean_min',
 'cA_std',
 'cA_std_max',
 'cA_std_min',
 'cD_energy',
 'cD_energy_max',
 'cD_energy_min',
 'cD_entropy',
 'cD_entropy_max',
 'cD_entropy_min',
 'cD_mean',
 'cD_mean_max',
 'cD_mean_min',
 'cD_std',
 'cD_std_max',
 'cD_std_min',
 'coeff_var',
 'd1_max',
 'd1_mean',
 'd2_max',
 'd2_mean',
 'hjworth_activity',
 'hjworth_complexity',
 'hjworth_mobility',
 'kurtosis',
 'skew']